This notebook has mutiple iterations but presenting the latest one

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/My Drive/Classroom/projects/Mercari")
!ls -l

total 7772369
-rw------- 1 root root        151 Nov 19 17:35  akarshan.1711@gmail.com_CS1.gdoc
-rw------- 1 root root     192263 Jan  2 21:08 'Copy of HptTfidf2.ipynb'
-rw------- 1 root root        151 Dec 16 13:22  EDA+FE.gdoc
-rw------- 1 root root    2441752 Dec 20 16:29  EDA.ipynb
-rw------- 1 root root      14393 Dec 27 21:06  FE+prep+modelling.ipynb
-rw------- 1 root root      30163 Dec 29 18:34  HptBrnandImpute.v1.0.ipynb
-rw------- 1 root root     249493 Jan  2 20:56  HptTfidf2.ipynb
-rw------- 1 root root     117022 Jan  2 21:18  HptTfidf.ipynb
-rw------- 1 root root  117131678 Jan  1 12:07  lgbt2.csv
-rw------- 1 root root   68399264 Jan  1 02:00  lgbt3.csv
-rw------- 1 root root     927353 Dec 28 15:17  mercari_mainV2.ipynb
-rw------- 1 root root     360448 Jan  2 21:18  Mercari_to3.db
-rw------- 1 root root      77824 Jan  2 14:07  Mercari_to4.db
-rw------- 1 root root     249856 Jan  2 20:56  Mercari_to5.db
-rw------- 1 root root     196608 Jan  2 21:08  Mercari_to6.db
-rw

In [3]:
#importing modules/libraries
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import sys
import os
import psutil
# from scipy.stats import randint as sp_randint
# from scipy.stats import uniform as sp_uniform


from tqdm.notebook import tqdm
# from collections import Counter
# from collections import defaultdict 
import re
import random
# from random import sample
# from bs4 import BeautifulSoup
import pickle
import inspect
import time

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.linear_model import Lasso,Ridge


# import string
# # import emoji
# # from wordcloud import WordCloud 
# import nltk
# nltk.download("stopwords")
# # nltk.download("brown")
# # nltk.download("names")
# # nltk.download('punkt')
# nltk.download('wordnet')
# # nltk.download('averaged_perceptron_tagger')
# # nltk.download('universal_tagset')
# # from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem.wordnet import WordNetLemmatizer
# # from nltk.stem.porter import PorterStemmer





import warnings
warnings.filterwarnings("ignore")

In [4]:
# defining root mean square error over Log transformed y_test data
# (as linear models homoscedasticity can be kept in check for better prediction) 
# and hence an effective Root Mean Square Log Error
def error(y_test, predictions): 
  return np.sqrt(mean_squared_error( y_test, predictions ))

In [5]:
tr_len = 1185329# demarkation of cv data(0.8 percent)
whole_tr = 1481661# whole train data

In [6]:
# loading prepared data
with open('tfidf.pickle','rb') as f:
  df=pickle.load(f)
with open('price_log.pickle','rb') as f:
  y=pickle.load(f)

In [7]:
df = df[:whole_tr] # only taking train and cv data
gc.collect()

150

In [8]:
np.isnan(df.data).sum()

0

In [9]:
df.shape

(1481661, 151063)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(df,y, train_size = round(0.8*df.shape[0]))

In [ ]:
#aplha over large range 
para = {'alpha': [0.001,0.01,0.1,1,2,5,10]}
clf= Ridge(max_iter=10000, tol=0.0005, solver='auto', random_state=34)
results = GridSearchCV(clf,para, cv=3, verbose=5,n_jobs=-1,scoring='neg_mean_squared_error',return_train_score=True)
results.fit(X_train,Y_train)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
print('Best score reached: {} with params: {} '.format(results.best_score_, results.best_params_))#gscv

In [ ]:
# zooming in the range from above parameter output this cell ahs been run multiple of times
# with different values one zooming more after another
para = {'alpha': np.linspace(4,6,5)}
clf= Ridge(max_iter=10000, tol=0.0005, solver='auto', random_state=34)
results = GridSearchCV(clf,para, cv=3, verbose=5,n_jobs=-1,scoring='neg_mean_squared_error',return_train_score=True)
results.fit(X_train,Y_train)

In [ ]:
print('Best score reached: {} with params: {} '.format(results.best_score_, results.best_params_))#gscv

In [ ]:
model= results.best_estimator_
# model= Ridge(alpha=4.5, max_iter=10000, tol=0.0005, solver='auto', random_state=34)

model.fit(X_train, Y_train)
Y_pred = model.predict(X_train)

print('train error {}'.format(error(Y_train,Y_pred)))
Y_pred = model.predict(X_test)

print('test error {}'.format(error(Y_test,Y_pred)))

In [ ]:
model= results.best_estimator_

model.fit(X_train, Y_train)
Y_pred = model.predict(X_train)

print('train error {}'.format(error(Y_train,Y_pred)))
Y_pred = model.predict(X_test)

print('test error {}'.format(error(Y_test,Y_pred)))

train error 0.41566154702658537
test error 0.4423676809467185


Below cells deals with hyperparameter optimization of LGBM models. They also have been run in parallel with numerous iterations on other notebooks, but presenting the latest one here. I had earlier tried Grid search in 2 stages where first i tunned hyperparametes dealing with complexity  of model like num of boosting rounds, child weight etc then tunned the convergence related parameters like learning rate and l1 and l2. That didnt work out well so i switched to optuna, Which is an alternative to sklearn's hyperopt as this has better convinience of API, visualizations, documentation and very important persistence and restarting after an instance crash of colab. Earlier i was tryin to optimize 12 to 13 hyperparametes of lgbm, but it showed results worse than ridge regression. So i went through the documenations and saw if lgbm overfits then only to hypertune all those parameters, my results were underfit. So i hypertuned only 4 to  5 parameters, and this is still going on its just one of the many copies.  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/Classroom/projects/Mercari")
!ls -l

!pip install optuna

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 7772062
-rw------- 1 root root        151 Nov 19 17:35 akarshan.1711@gmail.com_CS1.gdoc
-rw------- 1 root root        151 Dec 16 13:22 EDA+FE.gdoc
-rw------- 1 root root    2441752 Dec 20 16:29 EDA.ipynb
-rw------- 1 root root      14393 Dec 27 21:06 FE+prep+modelling.ipynb
-rw------- 1 root root      30163 Dec 29 18:34 HptBrnandImpute.v1.0.ipynb
-rw------- 1 root root     226201 Dec 31 14:25 HptTfidf2.ipynb
-rw------- 1 root root      41896 Jan  2 11:30 HptTfidf.ipynb
-rw------- 1 root root  117131678 Jan  1 12:07 lgbt2.csv
-rw------- 1 root root   68399264 Jan  1 02:00 lgbt3.csv
-rw------- 1 root root     282624 Dec 31 09:07 Mercari2.db
-rw------- 1 root root     282624 Dec 31 11:18 Mercari.db
-rw------- 1 root root     182982 Dec 31 21:24 mercari_lgb_tuned.py
-rw------- 1 root root     927353 Dec 28 15:17 mercari_mainV2.ipynb
-rw------- 1 root root  

In [ ]:
#https://optuna.readthedocs.io/en/stable/index.html

import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import pickle

import optuna



def obj_func(trial,data, target):

    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "n_estimator":trial.suggest_int("n_estimator", 80, 2500),
        "max_depth": trial.suggest_int("max_depth", 10,40),
        "num_leaves": trial.suggest_int("num_leaves", 80, 200),
        # "min_child_samples": trial.suggest_int("min_child_samples", 50, 300),
        # "min_child_weight" : trial.suggest_float('min_child_weight', 0.002, 1.0),
        "subsample_for_bin":trial.suggest_int("subsample_for_bin", 50000, 100000),
        "learning_rate" : trial.suggest_loguniform('learning_rate', 1e-1, 5),
        # "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-5, 10.0),
        # "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-5, 10.0),
        # "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.85),
        # "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.85),
        # "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        
        
    }
    tr_len= 1185329
    train_x, valid_x, train_y, valid_y = train_test_split(
                                      data,target,train_size = tr_len
                                      )
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

            
    

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
    gbm = lgb.train(
        param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
  
    
    return np.sqrt(sklearn.metrics.mean_squared_error(valid_y, pred_labels))


if __name__ == "__main__":

  with open('tfidf.pickle','rb') as f:
    data=pickle.load(f)
  with open('price_log.pickle','rb') as f:
    target=pickle.load(f)

    
    whole_len = 1481661
    data = data[:whole_len,:]
    target = target[:whole_len]


    study_name = 'Mercari'  
    study = optuna.create_study(study_name=study_name, 
                                storage='sqlite:///Mercari_to3.db',
                                load_if_exists=True,
                                pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize"
                                )


    study.optimize(lambda trial : obj_func(trial,data, target), n_trials=20 ,gc_after_trial=True)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-01-02 11:42:11,987] Using an existing study with name 'Mercari' instead of creating a new one.
[I 2021-01-02 11:52:06,905] Trial 13 finished with value: 0.550835330019086 and parameters: {'n_estimator': 290, 'max_depth': 30, 'num_leaves': 192, 'subsample_for_bin': 86578, 'learning_rate': 0.7556419396442797}. Best is trial 0 with value: 0.5502095808113284.
[I 2021-01-02 11:58:35,533] Trial 14 pruned. Trial was pruned at iteration 63.
[I 2021-01-02 12:00:33,854] Trial 15 pruned. Trial was pruned at iteration 10.
[I 2021-01-02 12:07:39,048] Trial 16 finished with value: 0.5470090793492802 and parameters: {'n_estimator': 337, 'max_depth': 18, 'num_leaves': 184, 'subsample_for_bin': 83326, 'learning_rate': 0.4398638878482992}. Best is trial 16 with value: 0.5470090793492802.
[I 2021-01-02 12:13:17,393] Trial 17 finished with value: 0.5520590552491569 and parameters: {'n_estimator': 1076, 'max_depth': 15, 'num_leaves': 98, 'subsample_for_bin': 79062, 'learning_rate': 0.37944470650238

Number of finished trials: 63
Best trial:
  Value: 0.5416196338607705
  Params: 
    learning_rate: 0.49559122346843787
    max_depth: 31
    n_estimator: 1298
    num_leaves: 101
    subsample_for_bin: 82534


In [ ]:
#https://optuna.readthedocs.io/en/stable/index.html

import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import pickle

import optuna



def obj_func(trial,data, target):

    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "n_estimator":trial.suggest_int("n_estimator", 2000, 2500),
        "max_depth": trial.suggest_int("max_depth", 30,42),
        "num_leaves": trial.suggest_int("num_leaves", 120, 200),
        # "min_child_samples": trial.suggest_int("min_child_samples", 50, 300),
        # "min_child_weight" : trial.suggest_float('min_child_weight', 0.002, 1.0),
        "subsample_for_bin":trial.suggest_int("subsample_for_bin", 50000, 100000),
        "learning_rate" : trial.suggest_loguniform('learning_rate', 1e-1, 1),
        # "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-5, 10.0),
        # "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-5, 10.0),
        # "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.85),
        # "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.85),
        # "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        
        
    }
    tr_len= 1185329
    train_x, valid_x, train_y, valid_y = train_test_split(
                                      data,target,train_size = tr_len
                                      )
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

            
    

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
    gbm = lgb.train(
        param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
  
    
    return np.sqrt(sklearn.metrics.mean_squared_error(valid_y, pred_labels))


if __name__ == "__main__":

  with open('tfidf.pickle','rb') as f:
    data=pickle.load(f)
  with open('price_log.pickle','rb') as f:
    target=pickle.load(f)

    
    whole_len = 1481661
    data = data[:whole_len,:]
    target = target[:whole_len]


    study_name = 'Mercari'  
    study = optuna.create_study(study_name=study_name, 
                                storage='sqlite:///Mercari_to3.db',
                                load_if_exists=True,
                                pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize"
                                )


    study.optimize(lambda trial : obj_func(trial,data, target), n_trials=20 ,gc_after_trial=True)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-01-02 15:28:09,390] Using an existing study with name 'Mercari' instead of creating a new one.
[I 2021-01-02 15:37:26,550] Trial 63 finished with value: 0.5412700356918323 and parameters: {'n_estimator': 2346, 'max_depth': 30, 'num_leaves': 122, 'subsample_for_bin': 84144, 'learning_rate': 0.5232593809983801}. Best is trial 63 with value: 0.5412700356918323.
[I 2021-01-02 15:46:45,782] Trial 64 finished with value: 0.5410045171521921 and parameters: {'n_estimator': 2372, 'max_depth': 30, 'num_leaves': 123, 'subsample_for_bin': 83697, 'learning_rate': 0.4917508722758207}. Best is trial 64 with value: 0.5410045171521921.
[I 2021-01-02 15:55:56,560] Trial 65 finished with value: 0.5451074998650021 and parameters: {'n_estimator': 2369, 'max_depth': 30, 'num_leaves': 123, 'subsample_for_bin': 84198, 'learning_rate': 0.6294799909717015}. Best is trial 64 with value: 0.5410045171521921.
[I 2021-01-02 16:05:43,884] Trial 66 finished with value: 0.5413544491340004 and parameters: {'n_es

Number of finished trials: 83
Best trial:
  Value: 0.5390072060426787
  Params: 
    learning_rate: 0.4168916591930659
    max_depth: 31
    n_estimator: 2294
    num_leaves: 151
    subsample_for_bin: 96101


In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['n_estimator', 'learning_rate'])

This cell is still running.

In [ ]:
#https://optuna.readthedocs.io/en/stable/index.html

import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import pickle

import optuna


def obj_func(trial,data, target):

    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "n_estimator":trial.suggest_int("n_estimator", 2000, 2500),
        "max_depth": trial.suggest_int("max_depth", 30,42),
        "num_leaves": trial.suggest_int("num_leaves", 120, 200),
        # "min_child_samples": trial.suggest_int("min_child_samples", 50, 300),
        # "min_child_weight" : trial.suggest_float('min_child_weight', 0.002, 1.0),
        "subsample_for_bin":trial.suggest_int("subsample_for_bin", 50000, 100000),
        "learning_rate" : trial.suggest_loguniform('learning_rate', 0.1, 0.5),
        # "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-5, 10.0),
        # "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-5, 10.0),
        # "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.85),
        # "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.85),
        # "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        
        
    }
    tr_len= 1185329
    train_x, valid_x, train_y, valid_y = train_test_split(
                                      data,target,train_size = tr_len
                                      )
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

            
    

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
    gbm = lgb.train(
        param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
  
    
    return np.sqrt(sklearn.metrics.mean_squared_error(valid_y, pred_labels))


if __name__ == "__main__":

  with open('tfidf.pickle','rb') as f:
    data=pickle.load(f)
  with open('price_log.pickle','rb') as f:
    target=pickle.load(f)

    
    whole_len = 1481661
    data = data[:whole_len,:]
    target = target[:whole_len]


    study_name = 'Mercari'  
    study = optuna.create_study(study_name=study_name, 
                                storage='sqlite:///Mercari_to3.db',
                                load_if_exists=True,
                                pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize"
                                )


    study.optimize(lambda trial : obj_func(trial,data, target), n_trials=20 ,gc_after_trial=True)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-01-02 18:29:21,802] Using an existing study with name 'Mercari' instead of creating a new one.
[I 2021-01-02 18:42:02,025] Trial 83 finished with value: 0.5377291210244696 and parameters: {'n_estimator': 2368, 'max_depth': 42, 'num_leaves': 151, 'subsample_for_bin': 99755, 'learning_rate': 0.4668527455862786}. Best is trial 83 with value: 0.5377291210244696.
[I 2021-01-02 18:52:28,839] Trial 84 finished with value: 0.5397384130004586 and parameters: {'n_estimator': 2369, 'max_depth': 32, 'num_leaves': 151, 'subsample_for_bin': 96346, 'learning_rate': 0.46649778950467075}. Best is trial 83 with value: 0.5377291210244696.
[I 2021-01-02 19:04:58,556] Trial 85 finished with value: 0.5369570866388655 and parameters: {'n_estimator': 2366, 'max_depth': 42, 'num_leaves': 152, 'subsample_for_bin': 99916, 'learning_rate': 0.4235203749209467}. Best is trial 85 with value: 0.5369570866388655.
[I 2021-01-02 19:17:33,639] Trial 86 finished with value: 0.5381498850308358 and parameters: {'n_e

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['n_estimator', 'learning_rate'])